# CS682 Final Project: Lane Detection using Computer Vision

## Team:
Abhishek Bodas - G01160204<br>
Aditya Indoori - G01129724<br>
Gaurav Bahl - G01057667<br>
Prashant Sahu - G01149803<br>
Pushkal Reddy - G01166539<br>

# Importing Libraries:

In [768]:
import numpy as np
import cv2
import os
import time
from matplotlib import pyplot as plt
from scipy.stats import norm, gamma, uniform 
import math  
import concurrent.futures

line_K = 5
delta = 5
numberOfParticles = 500

### Display images

In [769]:
def display_image(title, image):
    cv2.imshow(title, image) 
    cv2.waitKey(1)
    cv2.destroyAllWindows()

### Plot Images (Using matplotlib)

In [770]:
def plot_image(title, image):
    plt.imshow(image, cmap="gray")
    plt.title(title)
    plt.show()

### Reading Raw Data

In [771]:
def read_data():
    start_read_images_time = time.time()
    folder_root = './data_clr/data/'
    listOfImgs = []
    list_of_img_names = []
    for dirname, _, filenames in os.walk(folder_root):
        list_of_img_names.append(filenames)
        list_of_img_names = sorted(list_of_img_names[0])
        for name in list_of_img_names:
            if("png" in name):
                img = cv2.imread(folder_root+'/'+name)
                listOfImgs.append(img)

    end_read_images_time = time.time()
    time_taken_read_imgs = int(end_read_images_time-start_read_images_time)
    print("Number of images read: ", len(listOfImgs))
    print("Time taken to read images: ", time_taken_read_imgs,"seconds")
    print("Shape of each image: ",listOfImgs[0].shape)
    
    return listOfImgs

### Inverse Perspective Transform:
##### Picked 4 points from the image around the actual lane and converted it to see it as bird's eye view

In [772]:
def inverse_perspective_transform(img):
    dst_size = (img.shape[1],img.shape[0])
    src=np.float32([(270,360),(890,360),(660,210),(520,210)])
    dst=np.float32([(10,360), (1230, 360), (1230,10), (10,10)])
    Matrix = cv2.getPerspectiveTransform(src, dst)
    warped = cv2.warpPerspective(img, Matrix, dst_size)
    
    # Alternate to cv2.getPerspectiveTransform() is cv2.findHomography()
#     Matrix2 = cv2.findHomography(src, dst)
#     warped2 = cv2.warpPerspective(img, Matrix2[0], dst_size)
    
#     plot_image("Original Image", img)
#     plot_image("perspectiveTransform: ", warped)
    
    return warped

### Perspective Transform:
##### Converted back from the bird's eye view to actual image shape

In [773]:
def perspective_transform(img):
    dst_size = (img.shape[1],img.shape[0])
    dst=np.float32([(270,360),(890,360),(660,210),(520,210)])
    src=np.float32([(10,360), (1230, 360), (1230,10), (10,10)])
    Matrix = cv2.getPerspectiveTransform(src, dst)
    warped = cv2.warpPerspective(img, Matrix, dst_size)
    
#     plot_image("Detected Lane", img)
#     plot_image("Inverting Back to Original", warped)
    
    return warped

### Merging two images:
##### Merging original image with our result image that has two polylines showing lanes detected

In [774]:
def merge_images(image1, image2):
    alpha = 0.7
    beta = (1.0 - alpha)
    dst = cv2.addWeighted(image1, alpha, image2, beta, 0.0)
    
#     plot_image("Final Result", dst)    
    display_image("Result", dst)
    
    return dst

## Edge Detection and Distance Transform:
##### Detecting the edge using sobel in y direction. Then calculating the distance transform on that edge detected image

In [775]:
def edge_detect(warped):
    grayImg_warp = cv2.cvtColor(warped, cv2.COLOR_BGR2GRAY)
    thres_warp = cv2.threshold(grayImg_warp, 215, 255,cv2.THRESH_BINARY)[1]
    gauss_warp = cv2.GaussianBlur(thres_warp,(11,11),0)
    sobel_warp = cv2.Sobel(gauss_warp,cv2.CV_8U,1,0,ksize=3)

    # Calculating distance transform
    dst_tfm = cv2.distanceTransform(255-sobel_warp, cv2.DIST_L2, 5)
    
#     plot_image("Edge Detection", sobel_warp)
#     plot_image("Distance Transform", dst_tfm)
    
    return dst_tfm

# Partcle Filtering:

### Calculating actual left and right edge points from distance transform matrix: 

In [776]:
def get_actual_LR_points(arr_bt_row):
    lr = []
    left,right = 0,0
    mid_length = len(arr_bt_row)/2
    
    for i in range(1,len(arr_bt_row)-1):
        prev_pix_in = i-1
        next_pix_in = i+1
        next_diff = np.rint(arr_bt_row[i] - arr_bt_row[next_pix_in])
        prev_diff = np.rint(arr_bt_row[i] - arr_bt_row[prev_pix_in])
        if(arr_bt_row[i]<=10 and next_diff<=0 and prev_diff<=0):
            lr.append(i)

    central_number = 0 if len(list(filter(lambda k: k > mid_length, lr)))==0 else list(filter(lambda k: k > mid_length, lr))[0]
    central_index = lr.index(central_number) if central_number != 0 else len(lr)-1
    
    if central_index>0:
        left = sum(lr[:central_index])/len(lr[:central_index])
    if central_index!=len(lr)-1:
        right = sum(lr[central_index:])/len(lr[central_index:])

    return left,right

### Calculating the values of beta and gamma between actual lane points:

In [777]:
def get_beta(l, prev_particle):
    return math.atan((l-prev_particle[0])/delta)

def get_gamma(r, prev_particle):
    return math.atan((r-prev_particle[1])/delta)

### Generating random particles

In [778]:
def generate_random_particles(row_length, selected_particles):
    predicted_particles = np.array([])
    if(len(selected_particles) == 0):
        L = uniform(loc=0, scale=row_length/2).rvs(numberOfParticles) 
        R = uniform(loc=row_length/2+1, scale=row_length/2-1).rvs(numberOfParticles)
        B = np.zeros(numberOfParticles,dtype='uint8')
        G = np.zeros(numberOfParticles,dtype='uint8')
        predicted_particles = np.stack((L,R,B,G), axis=-1)
        
#         print("normal: ",predicted_particles)
#         predicted_particles = np.array(np.meshgrid(L,R,B,G)).T.reshape(-1,4)
#         print("new: ",predicted_particles)

    else:            
        num = int(numberOfParticles/len(selected_particles))
        for particle in selected_particles:
            L = uniform(loc=particle[0]-20, scale=40).rvs(num)
            R = uniform(loc=particle[1]-20, scale=40).rvs(num)
            B = uniform(loc=particle[2]-0.5, scale=1).rvs(num)
            G = uniform(loc=particle[3]-0.5, scale=1).rvs(num)
            
            if len(predicted_particles) == 0:
                predicted_particles = np.stack((L,R,B,G), axis=-1)
                
            else:
                predicted_particles = np.vstack((predicted_particles, np.stack((L,R,B,G), axis=-1)))

    return predicted_particles

### Calculating Gaussian weight (Not using anymore)

In [779]:
def gaussianWeight(S, mean, std):
#     return (1./(std*math.sqrt(2*math.pi)))*np.exp((-1/(2*(std)))*((S - mean)**2))
    
    val = np.divide(((S - mean)*(S-mean)),2*(std*std))
    print("val: ",val)

    left = np.divide(1,(std*math.sqrt(2*math.pi)))
    bracket = np.multiply(-1,val)
    right = math.exp(bracket)
    
    print("left: ",left, "\tright: ",right, "bracket: ",bracket)
    
    return left * right

## Get weights using Mean Squared Error (Currently using)

In [780]:
def get_weights_MSE(particles, prev_particle):
    weights = np.array([])
    for particle in particles:
        MSE = np.mean(np.square(np.subtract(prev_particle,particle)))
        weights = np.append(weights,1/MSE)
    normalized_weights = weights/np.amax(weights)
    
    return normalized_weights

### Get weights from slides (Not working: we are using MSE instead)

In [781]:
def get_weights(particles, dst_tfm, y):
    weights = []
    for particle in particles:
        dst_L = []
        dst_R = []
        dst_C = []
        for j in range(-line_K,line_K+1):
            x_L = int(particle[0] + j*math.sin(particle[2]))
            y_L = int(y + j*math.cos(particle[2]))
            x_R = int(particle[1] + j*math.sin(particle[3]))
            y_R = int(y + j*math.cos(particle[3]))
            dst_L.append(dst_tfm[y_L,x_L])
            dst_R.append(dst_tfm[y_R,x_R])
        
        print("dst_L: ",dst_L,"\ndst_R",dst_R)
        
        for j in range(-line_K,line_K+1):
            x_C = int(np.mean([particle[0],particle[1]]) + j*math.sin(np.mean([particle[2],particle[3]])))
            y_C = int(y + j*math.cos(np.mean([particle[2],particle[3]])))
            dst_C.append(dst_tfm[y_C,x_C])
    
        print("dst_C: ",dst_C)
              
        S_L = np.sum(np.abs(dst_L))
        S_R = np.sum(np.abs(dst_R))
        S_C =  abs(1/dst_C[line_K])
        mean_L = np.mean(dst_L)
        mean_R = np.mean(dst_R)
        mean_C = np.mean(dst_C)
        std_L = np.std(dst_L)
        std_R = np.std(dst_R)
        std_C = np.std(dst_C)
        
        print("S_R:",S_R,"\tmean_R",mean_R,"\tstd_R: ",std_R)
        g_L = gaussianWeight(S_L,mean_L,std_L)
        g_R = gaussianWeight(S_R,mean_R,std_R)
        g_C = gaussianWeight(S_C,mean_C,std_C)
        
        print("g_R:",g_R)
        
        w_dist = g_L * g_R
        w_center = g_C
        w_total = w_dist*w_center
        weights.append(w_total)
        
        print("weight inside: ",weights)
        break
        
    normalized_weights = weights/np.sum(weights)
    return normalized_weights

### Calculating the winning particles: 

In [782]:
def get_winning_particles(weights, particles):
    
    weights, particles = zip(*sorted(zip(weights, np.array(particles))))
    
    return np.array(particles)[-5:]

## Particle filter main function. It returns winning particles for the given row and image

In [783]:
def particle_filter(dst_tfm,row, prev_particle, selected_particles):
    start_time = time.time()
    y = row
    row_data = dst_tfm[y,:]
    
    # Calculating the actual left and right lane points using distance transform. These will be used to compare with our predicted points.
    l,r = get_actual_LR_points(dst_tfm[y,:])
    
    # If we can't find left or right point we will set the points based on previous row actual particle.
    if l == 0:
        if len(prev_particle)!=0:
            l = prev_particle[0]
            b = prev_particle[2]
        else:
            l=200
    if r == 0:
        if len(prev_particle)!=0:
            r = prev_particle[1]
            g = prev_particle[3]
#         else:
#             r=1100
    
    # Initializing the beta and gamma as 0 for first row. For later rows it will calculate the angle based on the new actual point and the previous actual point.
    if(row == dst_tfm.shape[0]-1):
        b = 0
        g = 0
    else:
        b = get_beta(l, prev_particle)
        if abs(b)>0.2:
            b=0
        g = get_gamma(r, prev_particle)
        if abs(g)>0.2:
            g=0
    
    # initializing the actual particle for this row
    prev_particle = np.array([l,r,b,g])
    
    before_particle = time.time()
    # generating random particles for this row
    particles = generate_random_particles(len(row_data), selected_particles) #IF BLANK
    after_particle = time.time()
    
#     ###Depreciated
#     weights = get_weights(particles, dst_tfm, y)
    
    # Calculating weights comparing the predicted particles with our actual particle for this row
    weights = get_weights_MSE(particles, prev_particle)
    after_weight = time.time()
    
    # Selecting the winning particles based on the weights
    selected_particles = get_winning_particles(weights, particles)
    after_selection = time.time()
    
#     print("particle calculation: ",after_particle-before_particle)
#     print("weights calculation: ",after_weight-after_particle)
#     print("selection calculation: ",after_selection-after_weight)
#     print("Total time taken for this row: ",after_selection - start_time)
    
#     print("selected_particles: ",selected_particles)
    return prev_particle, selected_particles

## Shadow Removal : Future Work

In [784]:
def remove_shadow(image):
    return

## Calculating and plotting the detected lane for the given image

In [785]:
def image_calculations(image, idx, prev_particle, selected_particles):
    start_time = time.time()
    # Inverting the perspective to bird's eye view
    warped = inverse_perspective_transform(image)
    dst_tfm = edge_detect(warped)
    
    # Image to store results for each frame
    test_image2 = np.zeros((dst_tfm.shape[0], dst_tfm.shape[1]), dtype = 'uint8')

    left_line_points = np.array([])
    right_line_points = np.array([])

    # starting the calculation from last row
    starting_row = dst_tfm.shape[0]-1
        
    
    # Calculating the particles for each row and storing the points to plot for left lane and right lane
    for row in range(starting_row,0,-delta):
        prev_particle, selected_particles = particle_filter(dst_tfm, row, prev_particle, selected_particles);
        
        # calculating the mean of the selected (winning) particles and setting left and right lane points
        mean_selected_particle = np.mean(selected_particles, axis=0)
        
        #setting left lane points
        if(int(mean_selected_particle[0])< dst_tfm.shape[1] and int(mean_selected_particle[0]) > 0):
            test_image2[row, int(mean_selected_particle[0])] = 255
            if len(left_line_points) == 0:
                left_line_points = np.array([int(mean_selected_particle[0]), row])
            else:
                left_line_points = np.vstack((left_line_points,np.array([int(mean_selected_particle[0]), row])))
        
        #setting right lane points
        if(int(mean_selected_particle[1]) < dst_tfm.shape[1] and int(mean_selected_particle[1]) > 0):
            test_image2[row, int(mean_selected_particle[1])] = 255
            if len(right_line_points) == 0:
                right_line_points = np.array([int(mean_selected_particle[1]), row])
            else:
                right_line_points = np.vstack((right_line_points,np.array([int(mean_selected_particle[1]), row])))
            
    
    # Changing the result image from grayscale to RGB to show detected lanes with different colors
    test_image2 = cv2.cvtColor(test_image2,cv2.COLOR_GRAY2RGB)
    
    # Plotting the lanes on the points stored in left and right array
    cv2.polylines(test_image2,[left_line_points],False,(0,255,255),35)
    cv2.polylines(test_image2,[right_line_points],False,(255,255,0),35)
        
    # Inverting the perspective back and merging with original image
    inverted = perspective_transform(test_image2)
    result = merge_images(image, inverted)
    
    end_time = time.time()
    print("Time taken for this image: ",end_time - start_time)
    
    return result,prev_particle,selected_particles

## Main function

In [786]:
def main():
    # Reading the images
    listOfImgs = read_data()
    start_time = time.time()
    
    # declaring size for the frames in the video for demo
    size = (listOfImgs[0].shape[1],listOfImgs[0].shape[0])
    
    # Initializing these here so that they can be carry forwarded to new frames with previous results
    prev_particle = np.array([])
    selected_particles = np.array([])
    
    # video writer for demo
    out = cv2.VideoWriter('project.avi',cv2.VideoWriter_fourcc(*'DIVX'), 5, size)
    
    # Calculating for each image in the given set
    for idx,image in enumerate(listOfImgs[:]):
        print("\nCalculating for image: ",idx+1)
        result,prev_particle,selected_particles = image_calculations(image,idx,prev_particle,selected_particles)
        out.write(result)
    
    # video release
    out.release()
    end_time = time.time()
    print("\nTotal time taken for ",len(listOfImgs)," images: ",end_time - start_time)
    return

In [787]:
main()

Number of images read:  297
Time taken to read images:  4 seconds
Shape of each image:  (375, 1242, 3)

Calculating for image:  1
Time taken for this image:  2.6090970039367676

Calculating for image:  2
Time taken for this image:  2.106325149536133

Calculating for image:  3
Time taken for this image:  2.3786590099334717

Calculating for image:  4
Time taken for this image:  1.9311811923980713

Calculating for image:  5
Time taken for this image:  2.037968158721924

Calculating for image:  6
Time taken for this image:  1.9518051147460938

Calculating for image:  7
Time taken for this image:  1.9203100204467773

Calculating for image:  8
Time taken for this image:  2.5588579177856445

Calculating for image:  9
Time taken for this image:  2.273467779159546

Calculating for image:  10
Time taken for this image:  2.069309949874878

Calculating for image:  11
Time taken for this image:  2.3276150226593018

Calculating for image:  12
Time taken for this image:  2.80436110496521

Calculating

KeyboardInterrupt: 